In [7]:
%pip install -q -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
import os
print(load_dotenv('../../.env'))
print(os.environ['LANGSMITH_PROJECT'])
os.environ['LANGSMITH_TRACING']="true"
os.environ['USER_AGENT'] = 'myagent'
QDRANT_URL="http://localhost:6333"

True
agentic-ops


## RAG APP

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

retriever = QdrantVectorStore(
            client=QdrantClient(url=QDRANT_URL),
            collection_name="documentations",
            embedding=OpenAIEmbeddings(model="text-embedding-3-small")
        ).as_retriever()

In [13]:
MODEL_NAME = "gpt-3.5-turbo" # "gpt-4o" "gpt-3.5-turbo", 
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

In [11]:
from langsmith import traceable
from openai import OpenAI
from typing import List

openai_client = OpenAI()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

# Experiment

In [12]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "test-dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

/Users/kartik/Documents/auto360/agents_course/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-1ff55466' at:
https://smith.langchain.com/o/140fee06-18bc-5f8b-b369-ecfa45c28bd3/datasets/c238d5fb-3c75-4f2d-a84a-e8c667dddd25/compare?selectedSessions=8eac833f-9413-425a-9853-a224adc56740




18it [00:57,  3.20s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Hugging accelerator and how do I use it?,Hugging Face Accelerate is a library designed ...,None,Hugging Face's Accelerate is a library designe...,1,5.027654,3e51eead-3f74-4b50-836d-2c9f270b8541,894761e2-d233-4210-99fb-1d6f54f772a7
1,How do I fine-tune a pre-trained model from Hu...,To fine-tune a pre-trained model from Hugging ...,None,To fine-tune a pre-trained model from Hugging ...,1,2.552916,10130a73-a232-4b34-b370-d37e6ddca4b6,bca83185-55d4-402a-90ce-3832bf30ea25
2,How do I use PEFT for efficient fine-tuning?,To use Parameter-Efficient Fine-Tuning (PEFT) ...,None,To use PEFT (Parameter-Efficient Fine-Tuning) ...,1,3.140116,3bf8162d-0619-41ee-b439-9347b8bf2885,33d58dfc-1d4b-44be-aa28-7a2a02604a6e
3,How do I create a Gradio demo for my Hugging F...,To create a Gradio demo for your Hugging Face ...,None,To create a Gradio demo for your Hugging Face ...,1,2.545520,8720b535-614d-4b4f-9b76-1334aa1e5ec8,ea51bfa0-3754-4aaa-9e31-9365023814bc
4,What is a model card and how do I create one?,A model card is a document associated with a m...,None,A model card is a documentation file providing...,1,2.521342,8fc42409-e3bb-4a30-bc7f-2d2ed748e465,18603791-efa1-476e-a4a1-c3b951d90696
5,How do I create a custom dataset with the Data...,To create a custom dataset with the Datasets l...,None,To create a custom dataset with the Hugging Fa...,1,4.011885,9da67a80-98c3-47ac-adb0-f7066b5fa192,8d2546b2-d78d-4500-b76d-050d5f57aa48
6,How do I quantize a model with bitsandbytes in...,To quantize a model using bitsandbytes in Hugg...,None,To quantize a model with bitsandbytes in Huggi...,1,3.947775,beeb1981-f60c-44d3-ad66-e880852f0b59,72d28a5b-f9c2-4dd5-9255-b5e74845f03f
7,How do I enable mixed precision training with ...,To enable mixed precision training with Huggin...,None,To enable mixed precision training with Huggin...,1,2.852316,bf8f4d54-4e79-410c-bbdb-a9dbf53161bc,de34277f-f2d4-413d-9c2f-5cce6284fc59
8,What is the Hugging Face Hub?,The Hugging Face Hub is a platform that allows...,None,The Hugging Face Hub is a platform for sharing...,1,1.904656,c8c05254-8878-4af1-94fe-677a4c023670,8e748293-f6fa-47e4-b5fa-31b5eedea979
9,How do I use Accelerate for distributed traini...,To use Accelerate for distributed training wit...,None,To use Accelerate for distributed training wit...,1,3.086425,df1aeb36-777c-4028-a809-0f9cb6ed50ee,a61268bc-9ca5-470e-a195-9e1a11dc68d1


## Modifying app

In [14]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-7a0d22c1' at:
https://smith.langchain.com/o/140fee06-18bc-5f8b-b369-ecfa45c28bd3/datasets/c238d5fb-3c75-4f2d-a84a-e8c667dddd25/compare?selectedSessions=27fd8261-b6a8-4f07-a781-7ae8c544f7db




18it [00:39,  2.20s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Hugging accelerator and how do I use it?,Hugging Face's Accelerate library helps users ...,None,Hugging Face's Accelerate is a library designe...,1,3.763830,3e51eead-3f74-4b50-836d-2c9f270b8541,041db74a-3181-4915-9d56-e5923cf4de5d
1,How do I fine-tune a pre-trained model from Hu...,To fine-tune a pre-trained model from Hugging ...,None,To fine-tune a pre-trained model from Hugging ...,1,1.737913,10130a73-a232-4b34-b370-d37e6ddca4b6,faed61a8-03b0-46fd-90f1-5c6216e1845e
2,How do I use PEFT for efficient fine-tuning?,I don't have information on how to use PEFT fo...,None,To use PEFT (Parameter-Efficient Fine-Tuning) ...,1,1.742571,3bf8162d-0619-41ee-b439-9347b8bf2885,6d8dee23-4a22-4f20-a047-f311a53573d5
3,How do I create a Gradio demo for my Hugging F...,I don't have information on creating a Gradio ...,None,To create a Gradio demo for your Hugging Face ...,1,1.004377,8720b535-614d-4b4f-9b76-1334aa1e5ec8,8b1ea574-82fe-4d9b-9639-cdde4c366630
4,What is a model card and how do I create one?,A model card is a document that provides detai...,None,A model card is a documentation file providing...,1,3.297387,8fc42409-e3bb-4a30-bc7f-2d2ed748e465,6f9bedd9-aa07-4e98-b98b-2c7834a25ca3
5,How do I create a custom dataset with the Data...,To create a custom dataset with the Datasets l...,None,To create a custom dataset with the Hugging Fa...,1,2.220911,9da67a80-98c3-47ac-adb0-f7066b5fa192,02ecf69e-674b-4502-95c7-b891ae340549
6,How do I quantize a model with bitsandbytes in...,To quantize a model with bitsandbytes in Huggi...,None,To quantize a model with bitsandbytes in Huggi...,1,3.151570,beeb1981-f60c-44d3-ad66-e880852f0b59,f4935dcd-77f5-4cbf-bb55-b15e29a4ca1d
7,How do I enable mixed precision training with ...,To enable mixed precision training with Huggin...,None,To enable mixed precision training with Huggin...,1,1.638544,bf8f4d54-4e79-410c-bbdb-a9dbf53161bc,6d50caee-5233-4bc3-82fc-6e4af4e8fd7b
8,What is the Hugging Face Hub?,The Hugging Face Hub is a platform where you c...,None,The Hugging Face Hub is a platform for sharing...,1,1.599100,c8c05254-8878-4af1-94fe-677a4c023670,67e9e0ed-479f-4a16-a644-30c516c373d8
9,How do I use Accelerate for distributed traini...,You can use Accelerate by installing it with `...,None,To use Accelerate for distributed training wit...,1,2.614899,df1aeb36-777c-4028-a809-0f9cb6ed50ee,34be303d-ea56-4e9b-b7ce-2043ab8f3a8c


## DEMO UI experiments
* UI evaluator run
* this local run
* the evaluation UI

## More settings

In [ ]:
# evaluate(
#     target_function,
#     data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
#     evaluators=[is_concise_enough],
#     experiment_prefix="initial dataset version"
# )
# evaluate(
#     target_function,
#     data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
#     evaluators=[is_concise_enough],
#     experiment_prefix="Crucial Examples split"
# )
# evaluate(
#     target_function,
#     data=client.list_examples(
#         dataset_name=dataset_name, 
#         example_ids=[   # We pass in a specific list of example_ids
#             # TODO: You will need to paste in your own example ids for this to work!
#             "",
#             ""
#         ]
#     ),
#     evaluators=[is_concise_enough],
#     experiment_prefix="two specific example ids"
# )

# evaluate(
#     target_function,
#     data=dataset_name,
#     evaluators=[is_concise_enough],
#     experiment_prefix="two repetitions",
#     num_repetitions=2   # This field defaults to 1
# )

# evaluate(
#     target_function,
#     data=dataset_name,
#     evaluators=[is_concise_enough],
#     experiment_prefix="concurrency",
#     max_concurrency=3,  # This defaults to None, so this is an improvement!
# )

# evaluate(
#     target_function,
#     data=dataset_name,
#     evaluators=[is_concise_enough],
#     experiment_prefix="metadata added",
#     metadata={  # We can pass custom metadata for the experiment, such as the model name
#         "model_name": MODEL_NAME
#     }
# )